# Módulo `search`
El módulo `search` contiene la implementación de la clase `Buscador` la cual expone una interfaz de alto nivel para desarrollar las consultas sobre lista de apariciones de las palabras que el `Tokenizer` ha delimitado en los documentos rastreados por el `Crawler` y entregados al `Control` para ser almacenados por los `Indexer`.
## Clase `Buscador`
Un `Buscador` precisa para su creación una referencia al `Control` que dirigirá el procesamiento de documentos y su posterior almacenamiento en los `Indexer`

In [1]:
%cd -q ..
from buscador.search import Buscador
from buscador.inverted_index import Control
mi_control = Control()
mi_buscador = Buscador(mi_control)

### Método `buscar()`
El método `buscar` expone una interfaz de alto nivel para el proceso de búsqueda de listas de apariciones de palabras en documentos. Se le debe pasar por parámetro un patrón de palabras el cual puede contener hasta un asterisco como comodín en el principio, medio o final del patrón. Se retorna un set de enteros identificadores de documentos. 

In [2]:
mi_control.procesar_documento("doc1", "cantor trotar")
mi_control.procesar_documento("doc2", "chatarra canción")
mi_control.procesar_documento("doc3", "ocación")
patrones = ["c*", "*cion", "c*tor"]
resultados = [mi_buscador.buscar(patron) for patron in patrones]
for indice, resultado in enumerate(resultados):
    print("Resultado del patrón '%s': %s" % (patrones[indice], resultado))

Resultado del patrón 'c*': {0, 1}
Resultado del patrón '*cion': {1, 2}
Resultado del patrón 'c*tor': {0}


El `Control` es el encargado de mantener un índice central que relaciona identificadores con direcciones de documentos.

In [3]:
resultados_direcciones = [[mi_control.documentos[indice] for indice in resultado] for resultado in resultados]
for indice, direcciones in enumerate(resultados_direcciones):
    print("Resultado del patrón '%s': %s" % (patrones[indice], direcciones))

Resultado del patrón 'c*': ['doc1', 'doc2']
Resultado del patrón '*cion': ['doc2', 'doc3']
Resultado del patrón 'c*tor': ['doc1']


### Métodos `interpreta_patron()` y `buscar_por_patron()`
El método `interpreta_patron()` devuelve una estructura en tupla que luego será utilizada por el método `buscar_por_patron()` para encontrar todas las palabras almacenadas en los índices que cumplan con el patrón.

In [4]:
for patron in patrones:
    print("El patrón '%s' se corresponde a la tupla: %s" % (patron, mi_buscador.interpreta_patron(patron)))

El patrón 'c*' se corresponde a la tupla: ('c', '*', '')
El patrón '*cion' se corresponde a la tupla: ('', '*', 'cion')
El patrón 'c*tor' se corresponde a la tupla: ('c', '*', 'tor')


In [5]:
for patron in patrones:
    print("El patrón '%s' se corresponde a las claves del índice: %s" 
          % (patron, mi_buscador.buscar_por_patron(mi_buscador.interpreta_patron(patron))))

El patrón 'c*' se corresponde a las claves del índice: ['cancion', 'cantor', 'chatarra']
El patrón '*cion' se corresponde a las claves del índice: ['ocacion', 'cancion']
El patrón 'c*tor' se corresponde a las claves del índice: ['cantor']


### Métodos `buscar_palabras()` y `buscar_palabras_reversas()`
Los métodos `buscar_palabras()` y `buscar_palabras_reversas()` son los utilizados por el método `buscar_por_patron()` para obtener las claves almacenadas en los `Indexer`. Ambas utilizan una palabra inicial inclusiva y otra final exclusiva para delimitar las claves almacenadas en un determinado rango alfabético.

In [6]:
mi_buscador.buscar_palabras("ch","t")

['chatarra', 'ocacion']

### Método `buscar_documentos()`
Este método es el utilizado por el método `buscar()` para obtener de los índices que mantiene el `Control`, la lista de identificadores de documentos en los que aparece la palabra pasada por parámetro.

In [7]:
mi_buscador.buscar_documentos("chatarra")

[1]

### Métodos `busqueda_conjuntiva()`, `busqueda_disjuntiva()` y `busqueda_negativa()`
Éstos métodos son métodos de agregación para simplificar las búsquedas iterativas sobre varios patrones.

In [8]:
for indice in mi_buscador.busqueda_negativa("*cion", mi_buscador.buscar("c*")):
    print(mi_control.documentos[indice])

doc1
